In [ ]:
import warnings
import os
import json  # If loading presaved indexes using freeze_model
import shap
import time

import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import RepeatedKFold, LeaveOneOut, GridSearchCV
from sklearn import neighbors, metrics
from dotenv import load_dotenv

from models.linear_regressions import Linear_reg
from scripts.model_actions import freeze_model

load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100


warnings.filterwarnings("ignore")

shap.initjs()


# <u>Tentative de modelisation et prédiction de la variable : Intensité d'utilisation energetique</u>

## <u>1 : Modelisations en prenant en compte la note Energy Star (E*)</u>
### <u>1.1 : Regressions lineaires</u>
### <u>1.2 : Potentielle méthode non lineaire : Regression KNN</u>

## <u>2 : Etude de l'importance de la note Energy Star</u>
### <u>2.1 : Modelisation sans utiliser de variables E* (E* Score ou E* certified)</u>
### <u>2.2 : Utilisation de la certification plutot que le score</u>

<hr>

<b><u>Important :</u></b>

_note : La modélisation prend en compte la variable "electricity", celle ci représentant la majorité de la consommation energétique, on peut considérer qu'il y a une fuite de données. Neanmoins, la facilité d'obtention des relevés de consommation electrique des batiments (i.e. requete a la compagnie electrique) justifie, à mon sens, son utilisation dans la modélisation. Si neanmoins cette variable est considérée comme trop informative, il est possible de l'enlever du modèle en utilisant la methode d'instance `regression.drop_col(["scaled_Electricity(kWh)"])`._

<hr>

## <u>1 : Modelisations prenant en compte toutes les variables retenues lors de l'étude</u>
### <u>1.1 : Regressions lineaires</u>

#### 1.1.a : Cross validation = Leave One Out
- On utilise dans un premier temps toutes les variables retenues lors de l'analyse exploratoire (breakpoint = export 1)
- Un split satisfaisant a deja été trouvé et fixé, ces données viendront sur-ecrire le split proposé par le modele
- On effectue 4 regressions (OLS, Ridge, Lasso et Elastic Net) avec les parametres par defaut de la classe pour la validation croisee : Leave One Out

In [ ]:
general_file = "./data/seattle_std_scaled.csv"  # Used as backup


In [ ]:
df_eui = pd.read_csv(general_file)
df_eui.set_index("OSEBuildingID", inplace=True)


In [ ]:
## Opening split

with open("./data/split_eui.json") as json_file:
    splits = json.load(json_file)


In [ ]:
ids_train = splits["train"]
ids_test = splits["test"]


In [ ]:
df_eui.head()


In [ ]:
df_eui.columns


In [ ]:
# Target : target_SourceEUI(kWh/m2) :

droplist = [
    "target_GHGEmissionsIntensity(kgCO2e/ft2)",  # Scaled target
    "scaled_SourceEUI(kWh/m2)",  # not to scale
    ]

df_model = df_eui.drop(columns=droplist)


In [ ]:
eui_target = "target_SourceEUI(kWh/m2)"
eui_regression = Linear_reg(dataframe=df_model, target=eui_target)


In [ ]:
# Forcing split : 

df_train_override = df_model[df_model.index.isin(ids_train)]
df_test_override = df_model[df_model.index.isin(ids_test)]


In [ ]:
eui_regression.force_split(
    df_test_ovr=df_test_override,
    df_train_ovr=df_train_override
)


In [ ]:
# Dropping the Cert (redundant)

eui_regression.drop_col(["EnergyStarCert"])


#### Execution : 
Parametres similaires à l'emission de GES, on ajustera au besoin: 
- Ridge = 0.1, 30, step 0.04
- Elastic Net = Alpha = Alpha_ridge = 0.1, 30, step 0.04, default L1 ratio
- Lasso Alpha = 0.01, 10, 0.02

In [ ]:
alphas_ridge = np.arange(0.1, 30, 0.04)
alphas_elnet = np.arange(0.1, 30, 0.04)
alphas_lasso = np.arange(0.01, 6, 0.01)

eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_lasso=alphas_lasso,
    alphas_ridge=alphas_ridge
)


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")

eui_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")

eui_regression.lasso_plot()


In [ ]:
metrics_l1out_estar = eui_regression.format_all_metrics()

metrics_l1out_estar


##### <u>Observations : 1.1.a :</u>
- Les modèles sont tous gloabalement performants.
- Le modele Ridge semble etre le plus performant sur les données d'entrainement avec une baisse du score R2 entre l'execution sur les donnees d'entrainement et sur les donnees de test.
- OLS est tres stable, on remarque des scores plus faibles que les autres regressions mais le modele est le seul a exceder les performaces attendues lors du passage au test (R2 plus haut, RMSE plus faible)
- ElasticNet semble etre le moins bon des modeles avec une RMSE de +|- 149 kWh/m2 sur le jeu de test

<hr>



#### 1.1.b : Cross validation = RepeatedKfold (10 folds aleatoires, 3 repetitions, i.e 30 folds)

- On utilise exactement les memes parametres que lors de la validation croisee utilisant Leave One Out
- On effectue 4 regressions (OLS, Ridge, Lasso et Elastic Net) avec les parametres par defaut de la classe pour la validation croisee
- Kfolds etant souvent beaucoup plus lourd a executer que Leave One Out, on attend de meilleurs scores mais des temps de traitement plus élevés

In [ ]:
## Changing CV to RepeatedKfold :
k_folds = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

eui_regression.common_parameters["cv"] = k_folds


In [ ]:
# Same split, same parameters, except cv = kfolds :

eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_lasso=alphas_lasso,
    alphas_ridge=alphas_ridge
)


In [ ]:
metrics_kfold_estar = eui_regression.format_all_metrics()


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")


In [ ]:
print("# Metriques des regressions utilisant Kfold #")
metrics_kfold_estar


In [ ]:
print("# Metriques des regressions utilisant Leave One Out #")
metrics_l1out_estar


##### Observations 1.1.b et comparaisons avec 1.1.a :

- Le temps d'execution est largement superieur, ce qui etait attendu
- La regression Ridge semble peut profiter de validation croisee par Kfold. Les performances sont extremement impressionantes sur le jeu d'entrainement (0.87 kWh/m2), mais il rentre dans les valeurs attendues (autour de 100kWh de RMSE) lors de son execution sur les donnees de test
- On peut questionner l'utilité de la validation par Kfold dans ce cas, meme si de tres faibles gains sont remarquables.
<br> 
<hr>
<br>
<b>Choix de Ridge Kfold :</b> <br>

- Le temps de fit est certes plus haut, mais une fois le modele entrainé, il predit les valeurs de test relativement precisement, et extremement rapidement.
- Elastic Net n'offre pas de gain significatif pour son temps d'entrainement largement superieur
- On analyse Ridge Kfold plus en details et c'est la regression principale qu'on utilisera pour comparer l'etape 1 à l'étape 2.

### <u>1.2 : Potentielle méthode non lineaire : Regression KNN</u>

Ici encore, le problème semble solvable en utilisant des methodes linéaires. Il est possible neanmoins d'appliquer, comme dans le cas de la modélisation sur les GàES, une regression par KNN pour verifier que d'autres methodes ne sont pas plus performantes.

In [ ]:
X_train, X_test = eui_regression.X_train, eui_regression.X_test
y_train, y_test = eui_regression.y_train, eui_regression.y_test

neighbors_candidates = [5, 7, 9, 11, 13, 15, 17, 19]

knnr = neighbors.KNeighborsRegressor()

# Grid search

l1out = LeaveOneOut()

param_grid_knnr = {'n_neighbors':neighbors_candidates}

score = "neg_mean_squared_error"

# KNN regressors Setup

knn_reg_l1o = GridSearchCV(
    estimator=knnr,
    param_grid=param_grid_knnr,
    cv=l1out,
    scoring=score
)

knn_reg_rkf = GridSearchCV(
    estimator=knnr,
    param_grid=param_grid_knnr,
    cv=k_folds,
    scoring=score
)


In [ ]:
knnr_l1o_fit_ts = time.perf_counter()

knn_reg_l1o.fit(
    X=X_train,
    y=y_train
)

knnr_l1o_fit_tf = time.perf_counter()

time_to_fit_knn_l1out = knnr_l1o_fit_tf - knnr_l1o_fit_ts

rmse_knnr_l1o = np.sqrt(abs(knn_reg_l1o.best_score_))
y_pred_train_l1o = knn_reg_l1o.predict(X_train)
r2_knnr_l1o = metrics.r2_score(y_true=eui_regression.y_train, y_pred=y_pred_train_l1o)


In [ ]:
print(f"RMSE Regression KNN, Cross Val : Leave one out = {rmse_knnr_l1o}")
print(f"Score R2 Regression KNN, Cross Val : Leave one out = {r2_knnr_l1o}")
print(f"Temps fit : {time_to_fit_knn_l1out}")
print(f"Meilleur hyperparametre : {knn_reg_l1o.best_params_}")


In [ ]:
knnr_rkf_fit_ts = time.perf_counter()

knn_reg_rkf.fit(
    X=X_train,
    y=y_train
)

knnr_rkf_fit_tf = time.perf_counter()

time_to_fit_knn_rkf = knnr_rkf_fit_tf - knnr_rkf_fit_ts

rmse_knnr_rkf = np.sqrt(abs(knn_reg_rkf.best_score_))
y_pred_train_rkf = knn_reg_rkf.predict(X_train)
r2_knnr_rkf = metrics.r2_score(y_true=eui_regression.y_train, y_pred=y_pred_train_l1o)


In [ ]:
print(f"RMSE Regression KNN, Cross Val : Repeated KFolds = {rmse_knnr_rkf}")
print(f"Score R2 Regression KNN, Cross Val : Repeated KFolds = {r2_knnr_rkf}")
print(f"Temps fit : {time_to_fit_knn_rkf}")
print(f"Meilleur hyperparametre : {knn_reg_l1o.best_params_}")


#### Observations sur KNN :
- Comme dans le cas des GàES, une methode de regression utilisant KNN n'est pas souhaitable. Le score R2 du modèle s'éffondre et l'erreur augmente grandement.
- Les modèles de regression linéaires semblent, dans ce cas et dans le cadre des modèles essayés, suffisamment precis et pertinents.
- Il n'est pas impossible qu'une autre méthode de regression non linéaire soit plus performante (Random Forest Regressor par exemple), mais le modèle Ridge validé par Repeated Kfolds se montre suffisamment satisfaisant pour justifier la non implémentation d'un méthode beaucoup plus lourde

## <u>2 : Etude de l'importance de la note Energy Star</u>


### <u>2.0 : Utilisation de SHAP</u> pour determiner l'importance des differentes variables sur la regression retenue lors de l'etape precedente :

- On identifiera quels sont les principales features qui entrent en compte lors de la modelisation
- On pourra voir si la variable Energy Star Score figure parmi les plus importantes features

In [ ]:
target_and_cert = [eui_target, "EnergyStarCert"]

X_all = eui_regression.df_origin.drop(columns=target_and_cert).to_numpy()
X100 = shap.utils.sample(X_all, 100) # 100 instances for use as the background distribution

features = eui_regression.df_origin.drop(columns=target_and_cert).columns

explainer = shap.LinearExplainer(eui_regression.ridge_cv, X100, feature_names=features)
shap_values = explainer(X_all)


In [ ]:
sample_ind = 18
my_waterfall = shap.plots.waterfall(shap_values[0], max_display=14, show=False)
my_waterfall.figure.set_size_inches(10, 10)
my_waterfall.figure.set_dpi(pc_dpi)

my_waterfall.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance")

plt.show()


In [ ]:
summary = shap.plots.beeswarm(shap_values, max_display=14, show=False)

summary.figure.set_size_inches(15, 8)
summary.figure.set_dpi(pc_dpi)

summary.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance")

plt.show()


#### Observations 2.0 : 

- Energy Star Score semble etre une valeur très importante au modèle : le plot summary (n2) montre que plus sa valeur est basse, plus l'intensité d'utilisation energetique augmente. Des valeurs hautes du score ("bons" scores) sont corrélés avec une valeur de sortie faible.

### <u>2.1 : Modelisation sans utiliser de variables E* (E* Score ou E* certified)</u>
### <u>2.2 : Utilisation de la certification plutot que le score</u>


In [ ]:
drop_col = ["ENERGYSTARScore", "EnergyStarCert"]

eui_regression.drop_col(col_list=drop_col)

eui_regression.common_parameters["cv"] = None


In [ ]:
eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")

eui_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")

eui_regression.lasso_plot()


In [ ]:
metrics_l1out_no_estar = eui_regression.format_all_metrics()

metrics_l1out_no_estar


In [ ]:
metrics_l1out_estar


#### heavy losses on all models
#### using kfold

In [ ]:
eui_regression.common_parameters["cv"] = k_folds


In [ ]:
eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")

eui_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")


In [ ]:
metrics_kfold_no_estar = eui_regression.format_all_metrics()

metrics_kfold_no_estar


In [ ]:
metrics_kfold_estar


#### heavy performances losses aswl

- Ici, contrairement a la modelisation de la prediction d'emission de GaES, la variable E* score semble très importante. On note de fortes baisses du score R2 de chaque modèle, et egalement une augmentation de l'erreur RMSE

#### Hypothèses :

- On sait que les batiments notés par EnergyStar sont qualifiés EnergyStar Certified s'ils ont une note superieure ou egale a 75.
- Est il possible de predire avec une haute confiance si un batiment aura une certification ou non ?
- Si oui, cette nouvelle statistique booleene aurait-elle un impact sur notre modele ? Si c'est le cas, il serait avantageux de tenter de predire ce score en utilisant des methode de classification. Cela permettrait d'avoir moins recours au calcul du score Per Se tout en retenant les benefices clairs apportés par cette variable

### 2.2 Utilisation de la certification :

In [ ]:
eui_regression.reset_cols()
eui_regression.drop_col(["ENERGYSTARScore"])


Same parameters, its the 3rd time, you know the drill

In [ ]:
# Let's fire up this bad boy : 
# Do me proud homes

eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")

eui_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")

eui_regression.lasso_plot()


In [ ]:
cert_metrics_l1out = eui_regression.format_all_metrics()

cert_metrics_l1out


In [ ]:
metrics_l1out_estar


In [ ]:
metrics_l1out_no_estar


In [ ]:
eui_regression.common_parameters["cv"] = k_folds


eui_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {eui_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {eui_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {eui_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {eui_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {eui_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.ridge_time_card.t_predict}")


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {eui_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {eui_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {eui_regression.lasso_time_card.t_predict}")


In [ ]:
cert_metrics_kfolds = eui_regression.format_all_metrics()

cert_metrics_kfolds


In [ ]:
cert_metrics_l1out


In [ ]:
metrics_kfold_estar


In [ ]:
metrics_kfold_no_estar


#### Observations :
- Les resultats sont encourageants. Il est evident que la certification, ou non, d'un batiment n'est pas aussi precise que son score mais on obtient un gain significatif par rapport aux modeles n'utilisant pas du tout le score EnergyStar.
- Si l'on garde l'exemple de Ridge, on obtient des resultats sensiblement plus optimistes. On perd certes en precision et on augmente l'erreur vis à vis du modele utilisant E* seul, mais cela peut etre un compromis interessant.
- On peut etudier l'importance de la feature avec la meme methode (shap) utilisee lors des precedents exemples. On s'attend a la voir contribuer substantiellement au modele.

In [ ]:
target_and_score = [eui_target, "ENERGYSTARScore"]

X_all_cert = eui_regression.df_origin.drop(columns=target_and_score).to_numpy()
X100 = shap.utils.sample(X_all_cert, 100) # 100 instances for use as the background distribution

features_cert = eui_regression.df_origin.drop(columns=target_and_score).columns

explainer = shap.LinearExplainer(eui_regression.ridge_cv, X100, feature_names=features_cert)
shap_values = explainer(X_all_cert)


In [ ]:
sample_ind = 18
my_waterfall = shap.plots.waterfall(shap_values[0], max_display=10, show=False)
my_waterfall.figure.set_size_inches(10, 10)
my_waterfall.figure.set_dpi(pc_dpi)

my_waterfall.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance, certification")

plt.show()


In [ ]:
summary = shap.plots.beeswarm(shap_values, max_display=14, show=False)

summary.figure.set_size_inches(15, 8)
summary.figure.set_dpi(pc_dpi)

summary.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance, certification")

plt.show()


### Observations sur l'hypothese et ouverture vers un autre probleme. Conclusion générale.

- L'hypothèse selon laquelelle la certification, ou non, d'un batiment apporte une contribution sembable en terme d'impact à celle de la Note Per Se.
- Les resultats sont particulierement interessants dans la mesure ou l'on passe d'une note sur 100 à une valeur binaire.
- Cependant, un autre probleme se pose : Nous avons pu etablir la certification des batiments en connaissant a priori leurs notes. L'idée est d'essayer de moins s'appuyer sur cette note - En clair, une nouvelle problematique emerge : Est il possible de predire si un batiment sera certifié ou non, en se basant sur les variables connues.
    - Dans le cas ou cela est possible est suffisamment precis, cette alternative est un bon compromis entre les pertes de performances consequentes dues à l'elimination totale d'E*.
    - Dans le cas ou cela est impossible ou trop peu precis. On recommandera, au possible, d'effectuer les calculs de ce score. La difference "avec/sans" est trop importante et le score beneficie bien trop au modele pour s'en passer.
- Dans l'ensemble, l'utilisation de Ridge, ici en utilisant Kfold (repetition 3 fois de 10 folds) est recommandée : c'est le modèle le plus precis même si quelque peu inconstant.